In [ ]:
import numpy as np
import astropy.units as u
import astropy.constants as con
import emcee
import corner
import speclite as speclite; from speclite import filters
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from chromatic import *
from scipy.optimize import minimize
from scipy.optimize import curve_fit


params = {'legend.fontsize': 'x-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('seaborn-deep')

## Calculate the scale height

In [ ]:
AB = 0.3 #bond albedo
Teff = 3600
#Teq = ((1-AB)*Teff**4)**(1/4) * u.K #600 * u.K 
Teq = 600 * u.K 
Rp = 4.15 * u.R_earth
Mp = 17 * u.M_earth # other listed values are 11 and 20 Me
mu = 10 # that's for an H2 atmosphere
transit_depth = 0.0512**2
deltaD_atm = (2*con.k_B*Teq*Rp*transit_depth) / (mu * con.m_p * con.G * Mp)

In [ ]:
one_scale_height = deltaD_atm.decompose()

In [ ]:
Rp = one_scale_height/transit_depth + 1
print(Rp)

## Calculate AU Mic's T$_{\rm{eff}}\pm\sigma$T$_{\rm{eff}}$ based on observed luminosity and radius

In [ ]:
L = np.random.normal(0.09,0.02,100000) # Plavchan+ 2009
R = np.random.normal(0.862,0.052,100000) # VLTI measurement
Teff = 5780.0*((L/(R**2))**(1./4.))

In [ ]:
np.median(Teff)

In [ ]:
np.percentile(Teff, [15.9, 50., 84.1])

## Calculate the Doppler-broadened linewidth, $\Delta\lambda$

In [ ]:
vsini = 8.0 * u.km/u.s # from vsini
lambda_i = np.linspace(0.55,0.89,100)*u.micron
delta_lambda = lambda_i * ( (vsini/con.c)/(1.0+(vsini/con.c)) )
delta_lambda = (delta_lambda.decompose()) * 1e9 * u.nm/u.m

plt.figure()
plt.plot(lambda_i,delta_lambda)
plt.xlabel(r'$\lambda_0$ ($\mu$m)')
plt.ylabel(r'$\Delta\lambda$ (nm)')

with Doppler broadening around 0.015-0.024 nm in the NRES bandpass, it is appropriate for us to continue binning the spectra to 0.05 nm so that our binsize accounts for doppler broadening